In [67]:
# importação das libs
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier


In [68]:
# input dos dados
dados = pd.read_csv("Dados Históricos - Ibovespa (2).csv")

In [69]:
dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,02.07.2025,139.051,139.586,140.049,138.384,"8,81B","-0,36%"
1,01.07.2025,139.549,138.855,139.695,138.855,"6,35B","0,50%"
2,30.06.2025,138.855,136.865,139.103,136.430,"7,68B","1,45%"
3,27.06.2025,136.866,137.113,137.209,136.469,"6,24B","-0,18%"
4,26.06.2025,137.114,135.767,137.353,135.756,"8,02B","0,99%"


In [70]:
# remove caractere % do nome da coluna
dados.columns = [col.strip().replace('.', '').replace('%', '').replace(' ', '_') for col in dados.columns]

In [71]:
dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol,Var
0,02.07.2025,139.051,139.586,140.049,138.384,"8,81B","-0,36%"
1,01.07.2025,139.549,138.855,139.695,138.855,"6,35B","0,50%"
2,30.06.2025,138.855,136.865,139.103,136.430,"7,68B","1,45%"
3,27.06.2025,136.866,137.113,137.209,136.469,"6,24B","-0,18%"
4,26.06.2025,137.114,135.767,137.353,135.756,"8,02B","0,99%"


In [72]:
# trata as colunas das cotações e converte as colunas númericas para float
for col in ['Último', 'Abertura', 'Máxima', 'Mínima']:
    dados[col] = dados[col].astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)


In [73]:
dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol,Var
0,02.07.2025,139051.0,139586.0,140049.0,138384.0,"8,81B","-0,36%"
1,01.07.2025,139549.0,138855.0,139695.0,138855.0,"6,35B","0,50%"
2,30.06.2025,138855.0,136865.0,139103.0,13643.0,"7,68B","1,45%"
3,27.06.2025,136866.0,137113.0,137209.0,136469.0,"6,24B","-0,18%"
4,26.06.2025,137114.0,135767.0,137353.0,135756.0,"8,02B","0,99%"


In [74]:
# Cria função para tratar a coluna de volume
def converter_volume(vol):
    vol = str(vol).upper()
    if 'B' in vol:
        return float(vol.replace('B', '').replace(',', '.')) * 1_000_000_000
    elif 'M' in vol:
        return float(vol.replace('M', '').replace(',', '.')) * 1_000_000
    elif 'K' in vol:
        return float(vol.replace('K', '').replace(',', '.')) * 1_000
    return pd.to_numeric(vol, errors='coerce')

In [75]:
dados['Vol'] = dados['Vol'].apply(converter_volume)
dados['Vol'] = dados['Vol'].fillna(dados['Vol'].median())

In [76]:
dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol,Var
0,02.07.2025,139051.0,139586.0,140049.0,138384.0,8.810000e+09,"-0,36%"
1,01.07.2025,139549.0,138855.0,139695.0,138855.0,6.350000e+09,"0,50%"
2,30.06.2025,138855.0,136865.0,139103.0,13643.0,7.680000e+09,"1,45%"
3,27.06.2025,136866.0,137113.0,137209.0,136469.0,6.240000e+09,"-0,18%"
4,26.06.2025,137114.0,135767.0,137353.0,135756.0,8.020000e+09,"0,99%"


In [77]:
# ordena data set por data crescente
dados['Data'] = pd.to_datetime(dados['Data'], dayfirst=True)
dados = dados.sort_values('Data').reset_index(drop=True)

In [78]:
dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol,Var
0,2023-07-03,119673.0,118092.0,119877.0,118092.0,8910000.0,"1,34%"
1,2023-07-04,119076.0,119673.0,119678.0,11883.0,6560000.0,"-0,50%"
2,2023-07-05,119549.0,119072.0,1202.0,118688.0,10960000.0,"0,40%"
3,2023-07-06,117426.0,119548.0,119548.0,117096.0,11030000.0,"-1,78%"
4,2023-07-07,118898.0,117427.0,119549.0,117427.0,10520000.0,"1,25%"


In [79]:
# Criar coluna de variação
dados['Variação'] = dados['Último'].diff().fillna(0)

In [80]:
dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol,Var,Variação
0,2023-07-03,119673.0,118092.0,119877.0,118092.0,8910000.0,"1,34%",0.0
1,2023-07-04,119076.0,119673.0,119678.0,11883.0,6560000.0,"-0,50%",-597.0
2,2023-07-05,119549.0,119072.0,1202.0,118688.0,10960000.0,"0,40%",473.0
3,2023-07-06,117426.0,119548.0,119548.0,117096.0,11030000.0,"-1,78%",-2123.0
4,2023-07-07,118898.0,117427.0,119549.0,117427.0,10520000.0,"1,25%",1472.0


In [81]:
# Novas features
dados['Var_pct'] = dados['Último'].pct_change().fillna(0)
dados['MM3'] = dados['Último'].rolling(window=3).mean().fillna(method='bfill')
dados['Range'] = dados['Máxima'] - dados['Mínima']

In [82]:
# Definir as categorias de variação
def classifica_variacao(x):
    if x > 0:
        return 'Subiu'
    elif x < 0:
        return 'Caiu'
    else:
        return 'Manteve'

dados['Alvo'] = dados['Variação'].apply(classifica_variacao)

In [83]:
dados.head()

,Data,Último,Abertura,Máxima,Mínima,Vol,Var,Variação,Var_pct,MM3,Range,Alvo
0,2023-07-03,119673.0,118092.0,119877.0,118092.0,8910000.0,"1,34%",0.0,0.000000,119432.666667,1785.0,Manteve
1,2023-07-04,119076.0,119673.0,119678.0,11883.0,6560000.0,"-0,50%",-597.0,-0.004989,119432.666667,107795.0,Caiu
2,2023-07-05,119549.0,119072.0,1202.0,118688.0,10960000.0,"0,40%",473.0,0.003972,119432.666667,-117486.0,Subiu
3,2023-07-06,117426.0,119548.0,119548.0,117096.0,11030000.0,"-1,78%",-2123.0,-0.017758,118683.666667,2452.0,Caiu
4,2023-07-07,118898.0,117427.0,119549.0,117427.0,10520000.0,"1,25%",1472.0,0.012536,118624.333333,2122.0,Subiu


In [84]:
# Defasar as features em 1 dia
features = ['Último', 'Abertura', 'Máxima', 'Mínima', 'Var_pct', 'MM3', 'Range', 'Vol']
for f in features:
    dados[f + '_lag1'] = dados[f].shift(1)

In [85]:
# Remover linhas iniciais com NaN
dados = dados.iloc[1:].reset_index(drop=True)

In [86]:
# # Remover colunas não usadas
# dados_modelo = dados.drop(columns=['Var', 'Data', 'Variação'] + features)

In [87]:
# Separar features e target
x = dados_modelo.drop(columns=['Alvo'])
y = dados_modelo['Alvo']

In [88]:
# Separar em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=7, shuffle=False)

In [89]:
# Otimização com GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

In [90]:
rf = RandomForestClassifier(random_state=7, class_weight='balanced')
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=0)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=7),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [5, 10, None],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]})

In [91]:
# Melhor modelo encontrado
best_rf = grid_search.best_estimator_
print(f"Melhores parâmetros encontrados: {grid_search.best_params_}\n")

Melhores parâmetros encontrados: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}



In [92]:
# Avaliação do modelo otimizado
y_predito = best_rf.predict(x_test)
print("Acurácia do Modelo Otimizado:", accuracy_score(y_test, y_predito))
print("\nClassification Report:\n", classification_report(y_test, y_predito, zero_division=0))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_predito))

Acurácia do Modelo Otimizado: 0.6266666666666667

Classification Report:
               precision    recall  f1-score   support

        Caiu       0.62      0.60      0.61        73
       Subiu       0.63      0.65      0.64        77

    accuracy                           0.63       150
   macro avg       0.63      0.63      0.63       150
weighted avg       0.63      0.63      0.63       150

Matriz de Confusão:
 [[44 29]
 [27 50]]


In [93]:
# Previsão dos próximos dias com o modelo otimizado
dias_a_prever = 10
ultimos_dados = dados.copy()
previsoes = []
simulados = []

In [94]:
# Preparar o último dado conhecido para iniciar a simulação
ultimo_dado_real = ultimos_dados.iloc[[-1]].copy()
proximo_ultimo = ultimo_dado_real['Último'].values[0]

In [95]:
for i in range(dias_a_prever):
    # Simular dados para o próximo dia
    variacao = random.uniform(-0.04, 0.04)
    proximo_ultimo = proximo_ultimo * (1 + variacao)
    abertura = proximo_ultimo * (1 + random.uniform(-0.015, 0.015))
    maxima = proximo_ultimo * (1 + abs(random.uniform(0, 0.03)))
    minima = proximo_ultimo * (1 - abs(random.uniform(0, 0.03)))
    var_pct = (proximo_ultimo - ultimos_dados.iloc[-1]['Último']) / ultimos_dados.iloc[-1]['Último']
    mm3 = np.mean([proximo_ultimo, ultimos_dados.iloc[-1]['Último'], ultimos_dados.iloc[-2]['Último']])
    range_ = maxima - minima
    vol = ultimos_dados['Vol_lag1'].median() * (1 + random.uniform(-0.1, 0.1))

    # Criar a linha com os dados defasados para a previsão
    nova_linha_features = pd.DataFrame({
        'Último_lag1': [proximo_ultimo],
        'Abertura_lag1': [abertura],
        'Máxima_lag1': [maxima],
        'Mínima_lag1': [minima],
        'Var_pct_lag1': [var_pct],
        'MM3_lag1': [mm3],
        'Range_lag1': [range_],
        'Vol_lag1': [vol]
    })

    # Fazer a predição
    pred = best_rf.predict(nova_linha_features)[0]
    previsoes.append(pred)
    simulados.append([proximo_ultimo, abertura, maxima, minima])

    # Adicionar a nova linha simulada ao histórico para a próxima iteração
    nova_linha_historico = {
        'Último': proximo_ultimo, 'Abertura': abertura, 'Máxima': maxima, 'Mínima': minima,
        'Var_pct': var_pct, 'MM3': mm3, 'Range': range_, 'Vol': vol,
        'Último_lag1': proximo_ultimo, 'Abertura_lag1': abertura, 'Máxima_lag1': maxima,
        'Mínima_lag1': minima, 'Var_pct_lag1': var_pct, 'MM3_lag1': mm3,
        'Range_lag1': range_, 'Vol_lag1': vol, 'Alvo': pred
    }
    
    ultimos_dados = pd.concat([ultimos_dados, pd.DataFrame([nova_linha_historico])], ignore_index=True)


In [96]:
print(f"\nPrevisão para os próximos {dias_a_prever} dias (usando modelo otimizado):")
for i, (p, s) in enumerate(zip(previsoes, simulados), 1):
    print(f"Dia {i}: {p} | Último Simulado: {s[0]:.2f} | Abertura: {s[1]:.2f} | Máxima: {s[2]:.2f} | Mínima: {s[3]:.2f}")


Previsão para os próximos 10 dias (usando modelo otimizado):
Dia 1: Caiu | Último Simulado: 143086.64 | Abertura: 142298.34 | Máxima: 146564.52 | Mínima: 138796.39
Dia 2: Subiu | Último Simulado: 138191.91 | Abertura: 139420.44 | Máxima: 140397.11 | Mínima: 135195.41
Dia 3: Caiu | Último Simulado: 132707.76 | Abertura: 134660.94 | Máxima: 135273.95 | Mínima: 131194.46
Dia 4: Subiu | Último Simulado: 132571.82 | Abertura: 134028.71 | Máxima: 135376.50 | Mínima: 129557.11
Dia 5: Subiu | Último Simulado: 133555.49 | Abertura: 132135.61 | Máxima: 136226.43 | Mínima: 131170.85
Dia 6: Subiu | Último Simulado: 131743.61 | Abertura: 131234.00 | Máxima: 133079.45 | Mínima: 127891.66
Dia 7: Subiu | Último Simulado: 130249.47 | Abertura: 130639.34 | Máxima: 130536.28 | Mínima: 129054.77
Dia 8: Caiu | Último Simulado: 130851.75 | Abertura: 132638.83 | Máxima: 132967.99 | Mínima: 128182.07
Dia 9: Caiu | Último Simulado: 134187.52 | Abertura: 133056.34 | Máxima: 138067.68 | Mínima: 131667.82
Dia 10